In [1]:
import importlib

import numpy as np

import rough_path_signatures_pricing.simulation_models as sm
from rough_path_signatures_pricing.signature_regressor import SignaturePricer

importlib.reload(sm)

Loaded intel_openmp/libiomp5md.dll from C:\Users\chris\Anaconda\envs\rough-path-signatures-pricing\Library\bin\libiomp5md.dll


<module 'rough_path_signatures_pricing.simulation_models' from 'C:\\Users\\chris\\VSCodeProjects\\rough-path-signatures-pricing\\src\\rough_path_signatures_pricing\\simulation_models.py'>

#### GBM Paths for Fitting Linear Functional with Signature to Approximate Payoffs

In [2]:
gbm_params = {"x0": 1, "t0": 0, "t1": 1, "n_steps": 100, "n_paths": 1000}
gbm_seed = 42

In [3]:
gbm = sm.GBM(mu=0, sigma=10, seed=gbm_seed)

In [4]:
gbm_paths, times, dW = gbm.simulate_paths(**gbm_params)
gbm_paths

array([[ 1.00000000e+00,  1.30471708e+00,  1.22737000e+00, ...,
         6.76500971e-14,  3.50879959e-14,  6.62306459e-14],
       [ 1.00000000e+00, -3.99841062e-02, -1.08242199e-02, ...,
         3.86802369e-10,  1.62807993e-10,  3.34984885e-10],
       [ 1.00000000e+00,  1.75045120e+00,  1.02493632e+00, ...,
        -2.85175202e-06, -2.27855633e-06, -4.61760783e-07],
       ...,
       [ 1.00000000e+00,  1.12117795e+00,  1.01854530e+00, ...,
        -2.11674771e-02, -1.17416113e-02, -2.93913265e-02],
       [ 1.00000000e+00,  1.13076419e+00,  5.30529875e-01, ...,
         2.63946628e-08,  2.10870672e-08,  1.78551360e-08],
       [ 1.00000000e+00,  1.82375313e+00,  5.86182090e+00, ...,
         1.95390428e-11,  3.21707230e-11,  5.32293803e-11]],
      shape=(1000, 101))

#### Fitting Linear Functional

In [5]:
N = 5


def asianPayoff(x: np.ndarray) -> np.ndarray:
    return np.mean(x, axis=1)

In [ ]:
bm_signature_price = SignaturePricer(signature_degree=N, payoff=asianPayoff)
bm_signature_price.fit(gbm_paths)

#### Creating Basket of Similar Options

In [10]:
hullwhite_params = {"x0": 1, "t0": 0, "t1": 1, "n_steps": 100, "n_paths": 100}
hullwhite_seed = 42


def theta(t):
    return 0.03

In [11]:
hullwhite = sm.HullWhite(a=1, sigma=10, theta=theta, seed=hullwhite_seed)

In [12]:
hullwhite_paths = hullwhite.simulate_paths(**hullwhite_params)
payoffs = hullwhite_paths